In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error

class LSTMModelPredictor:
    def __init__(self, train_data_path, new_data_path, feature_name):
        self.train_data_path = train_data_path
        self.new_data_path = new_data_path
        self.model = None
        self.scaler = MinMaxScaler()
        self.is_scaler_fitted = False
        self.feature_name = feature_name

    def load_data(self, path):
        data = pd.read_csv(path)
        return data

    def preprocess_data(self, data):
        features = data[[self.feature_name]]
        target = data['temp']  # Assuming 'temp' is the target for all scenarios
        scaled_features = self.scaler.fit_transform(features)
        return scaled_features, target

    def create_sequences(self, data, seq_length):
        sequences = []
        targets = []
        for i in range(len(data) - seq_length):
            sequences.append(data[i:i + seq_length])
            targets.append(data[i + seq_length])
        return np.array(sequences), np.array(targets)

    def build_model(self, seq_length):
        self.model = Sequential()
        self.model.add(LSTM(50, return_sequences=True, input_shape=(seq_length, 1)))
        self.model.add(LSTM(50))
        self.model.add(Dense(1))
        self.model.compile(optimizer='adam', loss='mean_squared_error')

    def train_model(self, X_train, y_train, epochs=20, batch_size=32, validation_split=0.2):
        history = self.model.fit(X_train, y_train, epochs=epochs, validation_split=validation_split, batch_size=batch_size)
        return history

    def evaluate_model(self, y_true, y_pred):
        mse = mean_squared_error(y_true, y_pred)
        return mse

    def predict_new_data(self, new_data, seq_length):
        scaled_new_features = self.scaler.transform(new_data)
        X_new = self.create_sequences(scaled_new_features, seq_length)
        new_predictions = self.model.predict(X_new)
        new_predictions = self.scaler.inverse_transform(new_predictions)
        new_predictions = new_predictions.squeeze()
        return new_predictions

    def run(self, seq_length=10, epochs=20, batch_size=32, validation_split=0.2):
        # Load training data
        train_data = self.load_data(self.train_data_path)
        # Preprocess training data
        scaled_features, target = self.preprocess_data(train_data)
        # Create sequences for LSTM
        X_train, y_train = self.create_sequences(scaled_features, seq_length)
        # Build LSTM model
        self.build_model(seq_length)
        # Train LSTM model
        self.train_model(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation_split)
        # Example of predicting on new data
        new_data = self.load_data(self.new_data_path)
        new_features = new_data[[self.feature_name]]
        # Predict on new data
        new_predictions = self.predict_new_data(new_features, seq_length)
        print(f'The next 60 days values predicted: {new_predictions[:60]}')
        # Return or do further processing with new_predictions if needed

class RenewableEnergyRecommendation:
    def __init__(self):
        self.renewable_recommendations = {
            ('clearsky', 'low'): 'Solar Photovoltaic (PV) Panels',
            ('clearsky', 'medium'): 'Solar Thermal Power Plants',
            ('clearsky', 'high'): 'Concentrated Solar Power (CSP) Plants',
            ('cloudy', 'low'): 'Wind Turbines',
            ('cloudy', 'medium'): 'Wind Farms',
            ('cloudy', 'high'): 'Offshore Wind Farms',
            ('rainy', 'low'): 'Hydroelectric Power Plants',
            ('rainy', 'medium'): 'Tidal Power Generators',
            ('rainy', 'high'): 'Wave Energy Converters',
            ('hot', 'low'): 'Geothermal Power Plants',
            ('hot', 'medium'): 'Geothermal Heat Pumps',
            ('hot', 'high'): 'Enhanced Geothermal Systems',
            ('cold', 'low'): 'Biomass Power Plants',
            ('cold', 'medium'): 'Biogas Digesters',
            ('cold', 'high'): 'Biofuel Production Facilities'
            # Add more conditions and recommendations as needed
        }

    def recommend_plant(self, weather_condition, wind_condition):
        key = (weather_condition, wind_condition)
        if key in self.renewable_recommendations:
            return self.renewable_recommendations[key]
        else:
            return "No recommendation available for the given conditions"


# Example usage:
if __name__ == "__main__":
    # Paths to your datasets
    train_data_path = 'bangaluru data.csv'
    new_temp_data_path = 'temppred.csv'
    new_wind_dir_data_path = 'wind_dir_pred.csv'
    new_wind_speed_data_path = 'wind_s_pred.csv'
    
    # Initialize and run the LSTMModelPredictor for temperature prediction
    lstm_predictor_temp = LSTMModelPredictor(train_data_path, new_temp_data_path, 'temp')
    lstm_predictor_temp.run(seq_length=10, epochs=20, batch_size=32, validation_split=0.2)

    # Initialize and run the LSTMModelPredictor for wind direction prediction
    lstm_predictor_wind_dir = LSTMModelPredictor(train_data_path, new_wind_dir_data_path, 'wind_direction')
    lstm_predictor_wind_dir.run(seq_length=10, epochs=20, batch_size=32, validation_split=0.2)

    # Initialize and run the LSTMModelPredictor for wind speed prediction
    lstm_predictor_wind_speed = LSTMModelPredictor(train_data_path, new_wind_speed_data_path, 'wind')
    lstm_predictor_wind_speed.run(seq_length=10, epochs=20, batch_size=32, validation_split=0.2)

renewable_rec = RenewableEnergyRecommendation()
print("""
        weather conditions:    
            clearsky: Clear skies, ideal for solar energy.
            cloudy: Partially cloudy weather.
            rainy: Rainy weather, suitable for hydroelectric power.
            hot: Hot weather, potential for geothermal energy.
            cold: Cold weather, can influence wind patterns.
        Wind Conditions:
            low: Low wind speed conditions.
            medium: Moderate wind speeds.
            high: High wind speeds, ideal for wind energy""")
predicted_weather_condition = input("enter the  weather condition")  # Example weather condition prediction
predicted_wind_condition = input("enter the  wind condition")  # Example wind condition prediction
recommendation = renewable_rec.recommend_plant(predicted_weather_condition, predicted_wind_condition)
print(f"Recommended renewable energy plant: {recommendation}")


Epoch 1/20


D:\New folder\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


314/314 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 0.0212 - val_loss: 0.0038
Epoch 2/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0046 - val_loss: 0.0035
Epoch 3/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0039 - val_loss: 0.0032
Epoch 4/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0034 - val_loss: 0.0027
Epoch 5/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0028 - val_loss: 0.0024
Epoch 6/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0026 - val_loss: 0.0024
Epoch 7/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0026 - val_loss: 0.0023
Epoch 8/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0026 - val_loss: 0.0031
Epoch 9/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0028 - val_loss: 0.0026
Epoch 10/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0026 - val_loss: 0.0023
Epoch 11/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0025 - val_loss: 0.0023
Epoch 12/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.

D:\New folder\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


314/314 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 0.0378 - val_loss: 0.0156
Epoch 2/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0133 - val_loss: 0.0125
Epoch 3/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0110 - val_loss: 0.0100
Epoch 4/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0100 - val_loss: 0.0103
Epoch 5/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0097 - val_loss: 0.0111
Epoch 6/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0097 - val_loss: 0.0097
Epoch 7/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0090 - val_loss: 0.0097
Epoch 8/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0098 - val_loss: 0.0096
Epoch 9/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0087 - val_loss: 0.0096
Epoch 10/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0090 - val_loss: 0.0095
Epoch 11/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0091 - val_loss: 0.0095
Epoch 12/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.

D:\New folder\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


314/314 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 0.0204 - val_loss: 0.0095
Epoch 2/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0078 - val_loss: 0.0063
Epoch 3/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0061 - val_loss: 0.0068
Epoch 4/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0061 - val_loss: 0.0061
Epoch 5/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0060 - val_loss: 0.0062
Epoch 6/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0063 - val_loss: 0.0062
Epoch 7/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0060 - val_loss: 0.0061
Epoch 8/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0059 - val_loss: 0.0061
Epoch 9/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0058 - val_loss: 0.0061
Epoch 10/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0059 - val_loss: 0.0060
Epoch 11/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0060 - val_loss: 0.0060
Epoch 12/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.

In [5]:
renewable_rec = RenewableEnergyRecommendation()
print("""
        weather conditions:    
            clearsky: Clear skies, ideal for solar energy.
            cloudy: Partially cloudy weather.
            rainy: Rainy weather, suitable for hydroelectric power.
            hot: Hot weather, potential for geothermal energy.
            cold: Cold weather, can influence wind patterns.
        Wind Conditions:
            low: Low wind speed conditions.
            medium: Moderate wind speeds.
            high: High wind speeds, ideal for wind energy""")
predicted_weather_condition = input("enter the  weather condition")  # Example weather condition prediction
predicted_wind_condition = input("enter the  wind condition")  # Example wind condition prediction
recommendation = renewable_rec.recommend_plant(predicted_weather_condition, predicted_wind_condition)
print(f"Recommended renewable energy plant: {recommendation}")



        weather conditions:    
            clearsky: Clear skies, ideal for solar energy.
            cloudy: Partially cloudy weather.
            rainy: Rainy weather, suitable for hydroelectric power.
            hot: Hot weather, potential for geothermal energy.
            cold: Cold weather, can influence wind patterns.
        Wind Conditions:
            low: Low wind speed conditions.
            medium: Moderate wind speeds.
            high: High wind speeds, ideal for wind energy


enter the  weather condition hot
enter the  wind condition low


Recommended renewable energy plant: Geothermal
